![Callysto.ca Banner](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-top.jpg?raw=true)



## Phidgets_4 Gauges.

Let's use the previous work and connect the Phidget temperature and humidity sensors to a live gauge in Plotly.

We use Plotly to get some nice looking gauges, and make them live by attaching them to the Phidget data. 

<h2 style="color: red">IMPORTANT</h2>
You MUST click the <span style="color:red">BIG, RED DISCONNECT BUTTON</span> at the end of this notebook when you are done. This tells the software to release the Phidget device, so it can be used by other notebooks you might try later.

## Set up gauges

We first set up the two gauges, for temperature and humidity

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

ind_hum = go.Indicator(
    mode = "gauge+number",
    value = 40,
    domain = {'x': [0, 1], 'y': [0, 1]},
    title = {'text': "Humidity"},
    gauge = {'axis': {'range': [0, 100]}}
)

ind_temp = go.Indicator(
    mode = "gauge+number",
    value = 20,
    domain = {'x': [0, 1], 'y': [0, 1]},
    title = {'text': "Temperature"},
    gauge = {'axis': {'range': [5, 35]}}
)


fig = make_subplots(
    rows=1,
    cols=2,
    specs=[[{'type' : 'domain'}, {'type' : 'domain'}]])
fig.append_trace(ind_temp, row=1, col=1)
fig.append_trace(ind_hum, row=1, col=2)

f = go.FigureWidget(fig)

f

## Setup the Phidget devices

You must be running on Chrome on your laptop or desktop, and the Phidget device needs to be connected to the USB port on your computer. 

As in the earlier notebooks, the main steps are thus:

0. Import some Python libraries
1. Import the Phidget library, in Javascript
2. Define and open the USB connection
3. Open the Humidity and Temperature devices
4. Read some values from the devices
5. When we are done, close the devices and USB connection

One change though is in Step 3, where we add some code that will communicate with Python. It looks like this:
```
    IPython.notebook.kernel.execute("f.data[1]['value'] = " + humidity );  
and
    IPython.notebook.kernel.execute("f.data[0]['value'] = " + temperature );  

```

These lines communicate with our Python code and update the values in data structure **f.data**.

Step 4 then becomes "live" automatically. 

So run the following Python and Javascript code, and connect your Phidget.

## Step 0.
Let's import a few Python libraries that we need. 

In [ ]:
import ipywidgets as widgets
from IPython.display import Javascript, display
from time import sleep

## Step 1.

Now import the Phidgets library. 

In [ ]:
%%js
requirejs.config({
    paths: { 
        'phidget22': ['https://unpkg.com/phidget22/browser/phidget22'], 
    },                                         
});
require(['phidget22'], (phidget22) => {
   window.phidget22 = phidget22; 
});

In [ ]:
## We pause for a second here, to allow the library to load
sleep(1)

## Step 2.
We now open a USB connection.  

In [ ]:
%%js

if (window.usbconn === undefined) {
    
    element.text("Creating a new USB Connection.");
    
    window.usbconn = new phidget22.USBConnection();
    
    usbconn.connect().then(() => {
        usbconn.requestWebUSBDeviceAccess();
    }).catch(err => {
        window.usbconn.delete();
        element.append("Error connecting to USB" + err);
    });
}

In [ ]:
## We rest for a bit while the USB connect
sleep(1)

### Confirm the connection 

In [ ]:
%%js
element.text(usbconn.connected);

## Step 3.

We make a request to open the humidity and temperature sensors. 

We connect them to the Plotly gauges using the call to kerner.execute. 

In [ ]:
%%js
window.humSensor = new phidget22.HumiditySensor();
humSensor.onHumidityChange = function (humidity) {
    IPython.notebook.kernel.execute(
        "f.data[1]['value'] = " + humidity
    );  
};
humSensor.open();

window.tempSensor = new phidget22.TemperatureSensor();
tempSensor.onTemperatureChange = function (temperature) {
    IPython.notebook.kernel.execute(
        "f.data[0]['value'] = " + temperature
    );  
};
tempSensor.open();

In [ ]:
## We rest for a bit while the devices connect.
sleep(1)

## Step 4. PAUSE HERE

The gauges should be running now. Everything is live.  Scroll up and check it out the gauges, to see the temperature and humitity values. Try breathing on the sensor, so you can see changes in the readings. 

We can even display a copy of the gauges right here, also live. 

In [ ]:
f

## Step 6. Closing down the sensors

It is **really important** to close the sensor now, as otherwise they will keep busy forever, always trying to update the sliders with the latest values. So, don't skip this next step.

The following cell creates a button that you can click to close the Phidgets. Click it once you are all done with the Phidgets in this notebook. 

In [ ]:
def doDisconnect(b):
    display(Javascript("""
        (async () => {
            await humSensor.close();
            await tempSensor.close();
            usbconn.close();
            usbconn.delete();
            delete window.usbconn;
            element.text("You have disconnected the Phidgets.");
        })();
    """))

run_button = widgets.Button(
    description = 'IMPORTANT: Click to disconnect', 
        button_style='danger',layout=widgets.Layout(width='50%', height='80px')
)
print("Press this button when you are done, to disconnect the Phidgets")
run_button.on_click(doDisconnect)

display(run_button)

### Confirm

You can confirm the phidget is open or closed by running the following cell. 

If it is still attached (true), try clicking the button above, again.

In [ ]:
%%js
element.text("Is the humidity sensor attached? " + humSensor.attached);
element.append("<p>Is the temperature sensor attached? " + tempSensor.attached);

## Conclusion

We have shown how to display the temperature and humidity values from the Phidget device using a live widget from the ipywidgets library. 

[![Callysto.ca License](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-bottom.jpg?raw=true)](https://github.com/callysto/curriculum-notebooks/blob/master/LICENSE.md)